<a href="https://colab.research.google.com/github/ellenwterry/PoliticalAnalysis/blob/main/Campaign_Planning_Models_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Hold for article

First, we'll load the program libraries *(nothing to see here unless you're running code)* BTW, I'm not really a python programmer, although I did write this code - I know it's not efficient or elegant - it's meant to be discussion support.

In [ ]:
# ---------- Load Libraries ---------- #

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from scipy import stats

!pip install nest-asyncio
import nest_asyncio
nest_asyncio.apply()

import patsy
from sklearn.linear_model import LogisticRegression

!pip install pystan
!pip install corner
import stan

import plotly.express as px
import plotly.graph_objects as go

!pip install geopy
from geopy.geocoders import Nominatim
import matplotlib.pyplot as plt
!pip install pygris
# import matplotlib.pyplot as plt
from pygris import core_based_statistical_areas
from pygris import tracts

from google.colab import files


import geopandas as gpd
import folium
# from google.colab import files


A synthetic dataset is stored on my github site - a very small sample, simulating a fact table that consolidates data from: party, census, secretary of state, canvassing feedback, and campaign survey sources *(campaigns can also buy datasets, like this but much better, from providers like https://targetsmart.com/)*

In [ ]:
# ---------- Get Data from Github ---------- #

url = 'https://raw.githubusercontent.com/ellenwterry/PoliticalAnalysis/main/VoteBase.csv'
VoteBase = pd.read_csv(url)

We begin by prepping the data for analysis. The dependent variable is "Support24" *(typical source: field/phonebank survey)*, and the independent variables are:   

> LastName, FirstName, Address, Sex, Age, Latitude / Longitude *(geocoded)*, Education *(can be party data, surveyed or inferred from census/PUMS)*, and HHIncome, ReligiousAffil , TopIssue, *(all from field/phonebank canvassing and/or surveys)*, and RRPetition *(a planned survey to be added later in this scenario)*       

In [ ]:
# ---------- Clean up data ---------- #

from sklearn import preprocessing
le = preprocessing.LabelEncoder()

le.fit(VoteBase['Sex'])

codes = {'NR':0, 'M': 1, 'F': 2}
VoteBase['Sex'] = VoteBase['Sex'].map(codes)

VoteBase['Age']=VoteBase.Age.astype('int32')

#VoteBase['LastPrimary'] = le.transform(VoteBase['LastPrimary'])
codes = {'NR':0, 'R': 1, 'D':2}
VoteBase['LastPrimary'] = VoteBase['LastPrimary'].map(codes)

#VoteBase['Education'] = le.transform(VoteBase['Education'])
codes = {'NR':0, 'HS': 1, 'Some College':2, 'Bachelor':3, 'Masters':4, 'Doctorate':5}
VoteBase['Education'] = VoteBase['Education'].map(codes)

#VoteBase['HHIncome'] = le.transform(VoteBase['HHIncome'])
codes = {'NR':0, 'Under 50k': 1, '50k-100k':2, '100k-200k':3, '200k-300k':4, '300k-500k':5, 'Over 500k':6}
VoteBase['HHIncome'] = VoteBase['HHIncome'].map(codes)

#VoteBase['ReligiousAffil'] = le.transform(VoteBase['ReligiousAffil'])
codes = {'NR':0,'Protestant': 1, 'Catholic':2, 'Other':3, 'None':4}
VoteBase['ReligiousAffil'] = VoteBase['ReligiousAffil'].map(codes)

#VoteBase['Support24'] = le.transform(VoteBase['Support24'])
codes = {'R':0, 'D': 1}
VoteBase['Support24'] = VoteBase['Support24'].map(codes)
# NOTE: NAs were excluded from sample so that algorithms could score using logistic scale - 2nd pass will use imputed values

#VoteBase['TopIssue'] = le.transform(VoteBase['TopIssue'])
codes = {'NR':0, 'RFree':1, 'Parents':2, 'Crime':3, 'Economy':4, 'Womens':5, 'Education':6, 'Environment':7, 'Democracy':8}
VoteBase['TopIssue'] = VoteBase['TopIssue'].map(codes)

# This is for the second data source (later)
codes = {'NS':0, 'NR':1,'Signed':2}
VoteBase['RRPetition'] = VoteBase['RRPetition'].map(codes)

We load the data into matrix / array structures that "Stan" likes *(https://mc-stan.org/)*

In [ ]:
# ---------- Create Matrices for Modeling ---------- #

yDf = VoteBase['Support24']
Xmatrix = patsy.dmatrix('Age + Sex + Education + HHIncome+ ReligiousAffil + LastPrimary + TopIssue', VoteBase)
rows = Xmatrix.shape[0]
columns = Xmatrix.shape[1]
rows, columns


(2498, 8)

# Model 1 *(Uninformed Priors)*
...and we build the first model. This is Bayesian logistic regression *(common in politics, sports, economics/business, insurance and banking)*. We're not really trying to predict votes or vote goals here - we're using the '24support' dependent variable to tune the independent variable associations, so that we can understand their effects *(or causes)* on outcomes, and help guide us in our voter contact processes. Yes, we can use this to set vote goals, but that's the beginning, not the end.

I hope this section is understandable to non-technical readers. Give it a try:

>The **data section** of the model sets up the data structure.

>The **parameters and transformed parameters** sections manage the parameters and create the equations we'll use *(regression in this case)*, and prepare the structures for iteration through observations *(it's really doing a lot of work for the model section)*

>The **model section** sets up the distributions *(bernoulli here to reflect the logistic equation)* to produce a target posterior distribution.

To review the Bayesian approach:

> ***posterior = data x prior***

If you look at **target += normal_lpdf(beta | p_b, p_sb);**, the **posterior is the target (distribution)**, **beta is the matrix of data** and **p_b and p_sb are the prior parameters with variances**. So, we actually are multiplying the data by the prior distribution parameters to get a posterior distribution. We won't know what the posterior distribution looks like until we sample it to define those parameters *(that's what the Stan program is doing for us - you can think of sampling like a random walk)*

**Why do we spend so much time on distributions?** Because, if we don't have a distribution, we don't have probability and credibility *(variance)*. If we don't have probability and credibility, we're making decisions on intuition *(not necessarily a bad thing - but I'd rather rely on evidence if I have it)*

In [ ]:
# ---------- Load Model 1 ---------- #

stanMod = """
data {
  int N_train;
  int K;
  int y_train[N_train];
  matrix[N_train, K] x_train;
  real p_b[K];
  real p_sb[K];
}
parameters {
  vector[K] beta;
}
transformed parameters {
  vector[N_train] y_hat;
  for(n in 1:N_train)
    y_hat[n] = x_train[n]*beta;
}
model {
  target += normal_lpdf(beta | p_b, p_sb);
  target += bernoulli_lpmf(y_train | inv_logit(y_hat));
}
"""

This is where we feed the data into the model.

I'm defining "uninformed priors" here - a common starting point if you have no experience with the data - so, an unbiased starting point: in the code **priors = np.repeat(0, 8)** I set all the pior parameters to 0, and with **priorsSigma = np.repeat(2, 8)**, I set the standard deviation *(variance, credibility)* of all the the parameters to 2, which is wide open for these parameters  - gives the sampler plenty of room to roam. We'll see what the data tells us:

In [ ]:
# ---------- Create Uninformed Priors ---------- #

priors = np.repeat(0, 8)
priorsSigma = np.repeat(2, 8)

mData = {
         "N_train": rows,
         "K": np.shape(Xmatrix)[1],
         "y_train": np.array(yDf),
         "x_train": Xmatrix,
          "p_b": priors,
          "p_sb": priorsSigma,
         }

Compile the model *(the stan program is running in C under the hood)*

In [ ]:
# Compile the Model
postr = stan.build(stanMod, data = mData, random_seed = 1)

...and run the sampler:

In [ ]:
# Muliply Prior and Data - Sample Prosterior
fit = postr.sample(num_chains=4, num_samples=1000)

OK, now we can pull the posterior parameters out. That's what we'll be working with from now on, and we can "tweak" the modeling process from here.  Also, notice the  statements:


>**def stable_sigmoid(x):
  return np.where(x >= 0, 1 / (1 + np.exp(-x)), np.exp(x) / (1 + np.exp(x)))**


>**stanProb = stable_sigmoid((np.dot(priors,Xmatrix.transpose())).transpose())**

This is what creates our probabilities - We use the parameters from the posterior  and run the data through the parameters. These statements are just a python version of the logistic equation:

>**𝑃(𝑌)=𝑒𝑥𝑝(𝛽0+𝛽𝑛𝑋𝑛)/𝑒𝑥𝑝(1+𝑒𝑥𝑝(𝛽0+𝛽𝑋𝑛)**

>Note: There are many advantages to generating predictions and probabilities outside of packages/programs/algorithms *(e.g., in high transaction, dynamic environments, analysts will often run analysis models in parallel with production models and pass parameters between the models to keep them relevant to the minute)*. There's also security and transaction management issues - which is why many IT shops wouldn't allow R or Python in production *(big, touchy subject outside of this discussion)*

Now that we have the probabilities *(of voting for the D candidate)*, we store those in a new "StanProb" column. I have also stored a theoretical vote *(discrete variable)* based on a 50% threshold *(against my better judgment)*: This is one of my pet peeves: Is there really a difference between a voter with a 49% overall likelihood of voting D vs. one with 51%? It's important to know why theres a 2% difference *(and that answer is in drivers and parameters)*. Obviously, we're going to want to talk to both *(never accept discrete variable assignments on face value)*

In [ ]:
# Get Parameters
df = fit.to_frame()
Params = df.describe().T
beta1 = Params['mean']['beta.1']
beta2 = Params['mean']['beta.2']
beta3 = Params['mean']['beta.3']
beta4 = Params['mean']['beta.4']
beta5 = Params['mean']['beta.5']
beta6 = Params['mean']['beta.6']
beta7 = Params['mean']['beta.7']
beta8 = Params['mean']['beta.8']
priors = np.array([beta1, beta2, beta3, beta4, beta5, beta6, beta7, beta8])

beta1Std = Params['std']['beta.1']
beta2Std = Params['std']['beta.2']
beta3Std = Params['std']['beta.3']
beta4Std = Params['std']['beta.4']
beta5Std = Params['std']['beta.5']
beta6Std = Params['std']['beta.6']
beta7Std = Params['std']['beta.7']
beta8Std = Params['std']['beta.8']
priorsSigma = np.array([beta1Std, beta2Std, beta3Std, beta4Std, beta5Std, beta6Std, beta7Std, beta8Std])

def stable_sigmoid(x):
  # Using np.where to avoid numerical overflow or underflow.
  return np.where(x >= 0, 1 / (1 + np.exp(-x)), np.exp(x) / (1 + np.exp(x)))

# Create Inferred Votes
stanProb = stable_sigmoid((np.dot(priors,Xmatrix.transpose())).transpose())


def updateInferredVote(x):
  if x >= .5:
    return 1
  else:
    return 0

VoteBase['StanProb'] = stanProb
VoteBase['StanVote'] = VoteBase['StanProb'].apply(updateInferredVote)


...and here's the parameters of Model 1's posterior distribution. First, keep in mind the order in the matrix: the Intercept *(garbage collection - i.e., what's left over)*, and 'Age + Sex + Education + HHIncome+ ReligiousAffil + LastPrimary + TopIssue'. So, based on the raw data, the model thinks that Education, HouseHold Income and Religious Affiliation are the most significant independents. That's all good, but there are a couple of problems with these parameters:

>1. Education and HHIncome are really too correlated *("no-no" in regression)* and should be transformed to a composite variable. But again, I'm not so much predicting outcomes as trying to understand associations, so I'm leaving them both in for now.

>2. We're interested in exploring issue effects here - starting with women's healthcare *(including reproductive freedom)*. The only variables addressing issues now, is TopIssue which is not great because the "Womens" selection was  recorded by a volunteer. And there's only one top issue. So, this is not an ideal driver. We also have "Sex" and we can suppose that women are more likely to support womens healthcare *(recognizing that evangelists will be opposed)*. We could estimate based on gender and religious affiliation, but outcomes are also influenced by Education, Income, LastPrimary, etc. So it's better to estimate within the model using the 'Sex' parameter, and wait for the survey on Reproductive Rights to be completed, and see how that works out.

We take the models parameters, and call them "priors" because we're going to turn around and use them again *(with some adjustment)* for the next model:

While we're discussing independent variables - there are many ways to visualize variable associations - I like box plots here. Looking at Religious Affiliation, and Top Issue:

In [ ]:
fig = px.box(VoteBase, x='ReligiousAffil', y='StanProb', points="all")
fig.update_layout(
    width=800,
    height=500,
    plot_bgcolor = "white",
    xaxis = dict(
        tickmode = 'array',
        tickvals = [0, 1, 2, 3, 4],
        title='Religious Affiliation',
        ticktext = ['NR', 'Protestant', 'Catholic', 'Other', 'None']
    ),
        yaxis=dict(title='Probability of Vote',),
)
fig.show()



In [ ]:
fig = px.box(VoteBase, x='TopIssue', y='StanProb', points="all")
fig.update_layout(
    width=800,
    height=500,
    plot_bgcolor = "white",
    xaxis = dict(
        tickmode = 'array',
        title='Top Issue',
        tickvals = [0, 1, 2, 3, 4, 5, 6, 7, 8],
        ticktext = ['NR', 'RFree', 'Parents', 'Crime', 'Economy', 'Womens', 'Education', 'Environment', 'Democracy']
    ),
        yaxis=dict(title='Probability of Vote',),
)
fig.show()


# Model 2 *(Adjusting Priors)*

Now that we have a good starting point for quantifying parameters, let's drill down a little more on the Women's healthcare effect. Our demo campaign has prepared a survey to ask voters to sign a petition to tell leaders that they support reproductive healthcare choices. We'll add that data when it's done, but for now, the campaign has to assign block turf and phonebanks. So, let's use a temp substitute to emphasize gender - like 'Sex' *(seems like 'Sex' should have discrete choices like  "Often" or "Seldom", doesn't it?)*. So, we'll use Sex and increase  parameter values from 0.11 to 0.4. The sampler will now start at 0.4 and wander back to 0.11 if we don't restrict it, so we squeeze the sigma *(the standard deviation was 0.12 after the first sample - we'll restrict it to .02 for now)* That should start increasing probabilities in the voters we want to talk with and start pushing them into groups designated for contact.

In [ ]:
Ipriors = priors.copy()
Ipriors[2] = 0.1
Ipriors[4] = 0.3
Ipriors[7] = 0.2
IpriorsSigma = priorsSigma.copy()
IpriorsSigma[2] = 0.02
IpriorsSigma[4] = 0.02
IpriorsSigma[7] = 0.02

# Emphasize Sex (temp), and TopIssue, De-emphasize HHIncome (which is overstated because of corelation with education and being a continous variable)

Reset the priors...

In [ ]:
mData = {
         "N_train": rows,
         "K": np.shape(Xmatrix)[1],
         "y_train": np.array(yDf),
         "x_train": Xmatrix,
          "p_b": Ipriors,
          "p_sb": IpriorsSigma,
         }


...and run the model:

In [ ]:
# Compile the Model
postr = stan.build(stanMod, data = mData, random_seed = 1)

In [ ]:
# Muliply Prior and Data - Sample Prosterior
fit = postr.sample(num_chains=4, num_samples=1000)

Get the parameters and compute probabilities *(if you look at the data now, you'll see that the probabilities increased with many voters - we'll see all this later)*

In [ ]:
# Get Parameters
df = fit.to_frame()
Params = df.describe().T
beta1 = Params['mean']['beta.1']
beta2 = Params['mean']['beta.2']
beta3 = Params['mean']['beta.3']
beta4 = Params['mean']['beta.4']
beta5 = Params['mean']['beta.5']
beta6 = Params['mean']['beta.6']
beta7 = Params['mean']['beta.7']
beta8 = Params['mean']['beta.8']
Ipriors = np.array([beta1, beta2, beta3, beta4, beta5, beta6, beta7, beta8])

beta1Std = Params['std']['beta.1']
beta2Std = Params['std']['beta.2']
beta3Std = Params['std']['beta.3']
beta4Std = Params['std']['beta.4']
beta5Std = Params['std']['beta.5']
beta6Std = Params['std']['beta.6']
beta7Std = Params['std']['beta.7']
beta8Std = Params['std']['beta.8']
IpriorsSigma = np.array([beta1Std, beta2Std, beta3Std, beta4Std, beta5Std, beta6Std, beta7Std, beta8Std])

def stable_sigmoid(x):
  # Using np.where to avoid numerical overflow or underflow.
  return np.where(x >= 0, 1 / (1 + np.exp(-x)), np.exp(x) / (1 + np.exp(x)))

# Create Inferred Votes
IstanProb = stable_sigmoid((np.dot(Ipriors,Xmatrix.transpose())).transpose())


def updateInferredVote(x):
  if x >= .5:
    return 1
  else:
    return 0

VoteBase['IStanProb'] = IstanProb
VoteBase['IStanVote'] = VoteBase['IStanProb'].apply(updateInferredVote)


And that will show an increase in probability to vote D and some of the voters in the middle are popping up in persuadable ranges.

In [ ]:
Xmatrix = patsy.dmatrix('Age + Sex + Education + HHIncome+ ReligiousAffil + LastPrimary + TopIssue + RRPetition', VoteBase)

rows = Xmatrix.shape[0]
columns = Xmatrix.shape[1]

# Model 3

OK, our demo campaign has finished the survey and we have the data. Now, we can add that to our model Having seen the voters that were pushed into the persuadable universe with our substitution priors in model 2, we can be more aggressive with the petition priors - the rationale here is that we're trying to capture some exogenous factors that will affect voter behavior *(e.g., Ohio)* and our data is deficient in doing that. Some aspects consider:

Increasing the Sex dimension boosted women voters, some of which popped into the persuadable universe *(we'll be quantifying that as >40% likelihood later in group assignments)*. Just being 'F' isnt' enough - the model looks at religion, income, last primary, etc. and balances the effect of all the variables with an emphasized Sex value. That's helpful, but there are many men who should be moving into the persuadable universe too, and the model won't pick that up.

Men, however, did have the opportunity to respond to the petition. So that will get some of those into the persuadables. However, there's a huge portion of voters *(men and women)* that don't respond to petitions, even if we did contact them. So again, it's reasonable to boost the priors here to emphasize this effect. Also, keep in mind that when we boost these variables, the model balances that will all the other variables, which will bring similar voters *(including men and who didn't respond)* into persuadable groups too.

So, this is where Bayesian modeling gives us the ability to apply experience and reason when we don't have the data, or the data are too difficult/expensive to acquire *(we could model the Ohio data, maybe create hierarchies, and gather priors for that - but would be a big effort and it's out of scope here)*. This is why we use Bayesian modeling in Business *(e.g., new markets with no data)*, or sports *(where the season is young and we don't have enough competitive, comparable games)*, or in banking *(where we have to set interest rates in markets where we don't yet know the risks)*. Data and computing is great - but billions of years of evolution has has produced organic thinking machines with intuition and the ability to process ambiguity, and it has proven to work very well.

Last time:

In [ ]:
priors2 = np.append(Ipriors[0:8], .4)
priorsSigma2 = np.append(IpriorsSigma[0:8], .02)
priorsSigma2[2] = 0.2 #open this up now that we have the petition

In [ ]:
mData = {
         "N_train": rows,
         "K": np.shape(Xmatrix)[1],
         "y_train": np.array(yDf),
         "x_train": Xmatrix,
          "p_b": priors2,
          "p_sb": priorsSigma2,
         }

In [ ]:
postr = stan.build(stanMod, data = mData, random_seed = 1)

In [ ]:
fit = postr.sample(num_chains=4, num_samples=1000)

pull the parameters and compute probabilities again:

In [ ]:
df = fit.to_frame()
Params = df.describe().T
beta1 = Params['mean']['beta.1']
beta2 = Params['mean']['beta.2']
beta3 = Params['mean']['beta.3']
beta4 = Params['mean']['beta.4']
beta5 = Params['mean']['beta.5']
beta6 = Params['mean']['beta.6']
beta7 = Params['mean']['beta.7']
beta8 = Params['mean']['beta.8']
beta9 = Params['mean']['beta.9']
priors2 = np.array([beta1, beta2, beta3, beta4, beta5, beta6, beta7, beta8, beta9])

beta1Std = Params['std']['beta.1']
beta2Std = Params['std']['beta.2']
beta3Std = Params['std']['beta.3']
beta4Std = Params['std']['beta.4']
beta5Std = Params['std']['beta.5']
beta6Std = Params['std']['beta.6']
beta7Std = Params['std']['beta.7']
beta8Std = Params['std']['beta.8']
beta9Std = Params['std']['beta.9']
priorsSigma2 = np.array([beta1Std, beta2Std, beta3Std, beta4Std, beta5Std, beta6Std, beta7Std, beta8Std, beta9Std])

def stable_sigmoid(x):
  # Using np.where to avoid numerical overflow or underflow.
  return np.where(x >= 0, 1 / (1 + np.exp(-x)), np.exp(x) / (1 + np.exp(x)))

# Create Inferred Votes
stanProb2 = stable_sigmoid((np.dot(priors2,Xmatrix.transpose())).transpose())

def updateInferredVote(x):
  if x >= .5:
    return 1
  else:
    return 0

VoteBase['StanProb2'] = stanProb2
VoteBase['StanVote2'] = VoteBase['StanProb2'].apply(updateInferredVote)

and here's the migration of parameters from the first to the third model:

In [ ]:
priors = np.append(priors, np.nan)
Ipriors = np.append(Ipriors, np.nan)

priorsSigma = np.append(priorsSigma, np.nan)
IpriorsSigma = np.append(IpriorsSigma, np.nan)

priorsDF = pd.DataFrame({"priors": priors, "priorsSigma": priorsSigma, "Ipriors": Ipriors, "IpriorsSigma": IpriorsSigma, "priors2": priors2, "priorsSigma2": priorsSigma2})

priorsDF


,priors,priorsSigma,Ipriors,IpriorsSigma,priors2,priorsSigma2
0,-4.608779,0.329315,-3.919789,0.161765,-3.979436,0.128638
1,-0.006715,0.003256,-0.004411,0.002122,-0.004422,0.001609
2,0.069156,0.114350,0.105275,0.019978,0.047830,0.080867
3,0.284556,0.052345,0.358541,0.035013,0.362764,0.028484
4,0.783808,0.081159,0.340060,0.018740,0.363792,0.017583
5,0.977975,0.055100,1.030467,0.037717,1.034660,0.029690
6,0.405648,0.102011,0.406445,0.070434,0.405128,0.058126
7,0.131926,0.026831,0.179525,0.015607,0.166438,0.013396
8,NaN,NaN,NaN,NaN,0.387693,0.019997


Another way to look at this migration, is to view the shift in probabilities. Some voters that you might not talk to initiatlly, have shifted into persuadable groups. Note how the bins in the lower probability shift down, and the bins in the middle, up and to the right.

In [ ]:
fig = go.Figure()
fig.add_trace(go.Histogram(x=VoteBase['StanProb'], marker_color='#DEDCDC', opacity=0.05, name = "Model 1"))
fig.add_trace(go.Histogram(x=VoteBase['IStanProb'], marker_color='#94CDD7', opacity=0.1, name = "Model 2"))
fig.add_trace(go.Histogram(x=VoteBase['StanProb2'], marker_color='#378796', opacity=1, name = "Model 3"))
fig.update_layout(barmode='overlay')
fig.update_traces(opacity=0.9, nbinsx=100)
fig.update_layout(
    autosize=False,
    width=800,
    height=400,
    plot_bgcolor = "white",
    xaxis=dict(title='Probability of Vote',),
)

fig.show()

We can bin the voters into groups to make it easier to make canvassing assignments. Here, we're cutting out voters below 40% probability, and then binning into Leaners (mostly indies), Base, and Leader groups:

In [ ]:
bins = [0, .4, .5, .9, 1]
labels = ['None', 'Leaners', 'Base', 'Leaders']
VoteBase['InitialGroup'] = pd.cut(x = VoteBase['StanProb'], bins = bins, labels = labels, include_lowest = False)
VoteBase['Group'] = pd.cut(x = VoteBase['StanProb2'], bins = bins, labels = labels, include_lowest = False)

In [ ]:
VoteBase.to_csv('VoteBaseCompare.csv')
files.download("VoteBaseCompare.csv")
priorsDF.to_csv("PriorsDF.csv")
files.download("PriorsDF.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Create Interest Groups

In [ ]:
# Assign to Interest Groups (just 2 issues here)

WomensBase = VoteBase.loc[((VoteBase['Group'] == 'Leaders') | (VoteBase['Group'] == 'Base'))& (VoteBase['TopIssue'] == 5)]
EnvironmentBase = VoteBase.loc[(VoteBase['Group'] == 'Base') & (VoteBase['TopIssue'] == 7)]
WomensBase = WomensBase.reset_index()
EnvironmentBase = EnvironmentBase.reset_index()



Interest Groups are networking structures to engage the base and the indie leaners. Most likely, campaigns will want to recruit these groups by phone and email *(finding leaders first, and letting them build the group)*. The groups could then be integrated with canvassing to address specific issues in the field:

In [ ]:
m = folium.Map(width=1000,height=800,location=[41.07, -73.6], zoom_start=12)

for i in range(VoteBase[VoteBase.columns[0]].count()):
  folium.CircleMarker(
    [VoteBase['Latitude'][i], VoteBase['Longitude'][i]], radius = 1, weight = 2, color = "gray").add_to(m)

for i in range(WomensBase[WomensBase.columns[0]].count()):
  folium.CircleMarker(
    [WomensBase['Latitude'][i], WomensBase['Longitude'][i]], radius = 2, weight = 3, color = "red").add_to(m)

for i in range(EnvironmentBase[EnvironmentBase.columns[0]].count()):
  folium.CircleMarker(
    [EnvironmentBase['Latitude'][i], EnvironmentBase['Longitude'][i]], radius = 3, weight = 2, color = "green").add_to(m)

m

Finally, if you're wondering how many voters moved up groups from model 1 to model3, the number is 88:

In [ ]:
# how many voters moved into focus groups, or moved up a group within focus groups?

# rate groups by prob:
codes = {'None':0, 'Grp 1':1, 'Grp 2': 2, 'Grp 3':3, 'Grp 4':4, 'Grp 5':5, 'Grp 6':6}
VoteBase['IGrpNo'] = VoteBase['InitialGroup'].map(codes)
VoteBase['GrpNo'] = VoteBase['Group'].map(codes)

In [ ]:
VoteBase['StanVote2'].sum()

1363

Summing up the vote goal issue: If we used a 50% probability to assign a D vote, we'd predict 1,287 just running a plain logistic model *(I also ran the Scikit-learn logistic model and got the same thing as the first model)*.

In this demo precinct of 2500, we would typically set a vote goal at 50% + 5% cushion, so 1,375 would be the target. The final model predicted 1,309 votes, but we now have groups of persuadables to contact that wouldn't have shown up - and based on the confounders at play, it seems like these persuadables are a good target. It's going to be tight, but winnable!      

Thanks for reading and please let me know of any comments or questions.

In [ ]:
FinalBox = VoteBase = VoteBase.loc[VoteBase['TopIssue'] > 2]


fig = px.box(FinalBox, x='TopIssue', y='StanProb2', points="all")

fig.add_shape(type="rect", x0=2.5, y0=.4, x1=8.5, y1=.5, fillcolor="#F47811", opacity=0.3)
fig.add_shape(type="rect", x0=2.5, y0=.5, x1=8.5, y1=.6, fillcolor="#33F411", opacity=0.3)
fig.add_shape(type="rect", x0=2.5, y0=.6, x1=8.5, y1=.7, fillcolor="#1159F4", opacity=0.3)
fig.add_shape(type="rect", x0=2.5, y0=.7, x1=8.5, y1=.8, fillcolor="#A811F4", opacity=0.3)
fig.add_shape(type="rect", x0=2.5, y0=.8, x1=8.5, y1=.9, fillcolor="#F411BE", opacity=0.3)
fig.add_shape(type="rect", x0=2.5, y0=.9, x1=8.5, y1=1,  fillcolor="#F4114A", opacity=0.3)


fig.update_layout(
    width=800,
    height=500,
    plot_bgcolor = "white",
    xaxis = dict(
        tickmode = 'array',
        title='Top Issue',
        tickvals = [3, 4, 5, 6, 7, 8],
        ticktext = ['Crime', 'Economy', 'Womens', 'Education', 'Environment', 'Democracy']
    ),
        yaxis=dict(title='Probability of Vote',),
)

fig.show()
